In [1]:
%reload_ext autoreload
%autoreload 2

In [1]:
import os
import pickle
import itertools

import tensorflow as tf
import numpy as np
from pprint import pprint

from loader import load_sentences, update_tag_scheme, load_or_create_maps
from loader import char_mapping, tag_mapping, augment_with_pretrained
from loader import prepare_dataset

from model import Model

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\zjuwa\AppData\Local\Temp\jieba.cache
Loading model cost 0.738 seconds.
Prefix dict has been built succesfully.


In [2]:
# flags = tf.app.flags
# flags.DEFINE_string("train_file",   os.path.join("../CEC-Corpus/news.train"),  "Path for train data")
# flags.DEFINE_string("dev_file",     os.path.join("../CEC-Corpus/news.dev"),    "Path for dev data")
# flags.DEFINE_string("test_file",    os.path.join("../CEC-Corpus/news.test"),   "Path for test data")
# FLAGS = tf.app.flags.FLAGS
config = {}

config['crf'] = True # "Use CRF"
config['clean'] = False # "clean train folder"
config['train'] = True # "weither train the model"

config['train_file'] = os.path.join("../CEC-Corpus/news.train")
config['dev_file'] = os.path.join("../CEC-Corpus/news.dev")
config['test_file'] = os.path.join("../CEC-Corpus/news.test")

config['map_file'] = 'maps.pkl' # "file for maps"
config['emb_file'] = 'wiki_100.utf8' # "Path for pre_trained embedding"

config['pre_emb'] = True # "Wither use pre-trained embedding"
config['zeros'] = False  # "Wither replace digits with zero"
config['lower'] = True   # "Wither lower case"

config['seg_dim'] = 20 # "Embedding size for segmentation, 0 if not used"
config['char_dim'] = 100 # "Embedding size for characters"
config['lstm_dim'] = 200 # "Num of hidden units in LSTM"
config['tag_schema'] = 'iobes' # "tagging schema iobes or iob"

config['lr'] = 0.001 # "Initial learning rate"
config['batch_size'] = 64 # "Batch size"
config['optimizer'] = 'adam' # "Optimizer for training"
config['clip'] = 5 # "Gradient clip"
config['dropout'] = 0.5 # "Dropout rate"

# load sentences and update them to format we want
train_sentences = load_sentences(config['train_file'], config['lower'], config['zeros'])
dev_sentences = load_sentences(config['dev_file'], config['lower'], config['zeros'])
test_sentences = load_sentences(config['test_file'], config['lower'], config['zeros'])
update_tag_scheme(train_sentences, config['tag_schema'])
update_tag_scheme(test_sentences, config['tag_schema'])
# load or create maps
char_to_id, id_to_char, tag_to_id, id_to_tag = load_or_create_maps(train_sentences, test_sentences, config)

config['num_tags'] = len(tag_to_id)
config['num_chars'] = len(char_to_id)

In [3]:
# prepare data, get a collection of list containing index
# data = [[chars], [idx_chars], [segments], [idx_tag]]
# where segments is {0: word with one sigle char, 1: begin of a word, 2: inside a word, 3: end of a word}
train_data = prepare_dataset(
    train_sentences, char_to_id, tag_to_id, config['lower']
)
dev_data = prepare_dataset(
    dev_sentences, char_to_id, tag_to_id, config['lower']
)
test_data = prepare_dataset(
    test_sentences, char_to_id, tag_to_id, config['lower']
)
print("%i / %i / %i sentences in train / dev / test." % (
    len(train_data), 0, len(test_data)))

2977 / 0 / 1488 sentences in train / dev / test.


In [4]:
# make path for store log and model if not exist
# make_path(FLAGS)
# if os.path.isfile(FLAGS.config_file):
#     config = load_config(FLAGS.config_file)
# else:
#     config = config_model(char_to_id, tag_to_id)
#     save_config(config, FLAGS.config_file)
# make_path(FLAGS)

# log_path = os.path.join("log", FLAGS.log_file)
# logger = get_logger(log_path)
# print_config(config, logger)

# limit GPU memory
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
model = Model(config)
model.set_dataset(train_data, 'train_data')
model.set_dataset(test_data, 'test_data')
model.set_dataset(dev_data, 'dev_data')

c:\users\zjuwa\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\gradients_impl.py:97: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [5]:
train_init = model.make_dataset_init('train_data', shuffle=3000)

64


In [6]:
with tf.Session(config=tf_config) as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(100):
        sess.run(train_init)
        loss = []
        n_batches = 0
        try:
            while True:
                step, batch_loss = model.run_step(sess, True)
                loss.append(batch_loss)
                n_batches += 1
                if step % 100 == 0:
                    print("step:{} loss:{:>9.6f}".format(
                         step, np.mean(loss)))
        except tf.errors.OutOfRangeError:
            pass
        print(i)

# with tf.Session(config=tf_config) as sess:
#     model = create_model(sess, Model, FLAGS.ckpt_path, load_word2vec, config, id_to_char, logger)
#     logger.info("start training")
#     loss = []
#     for i in range(100):
#         for batch in train_manager.iter_batch(shuffle=True):
#             step, batch_loss = model.run_step(sess, True, batch)
#             loss.append(batch_loss)
#             if step % FLAGS.steps_check == 0:
#                 iteration = step // steps_per_epoch + 1
#                 logger.info("iteration:{} step:{}/{}, "
#                             "NER loss:{:>9.6f}".format(
#                     iteration, step%steps_per_epoch, steps_per_epoch, np.mean(loss)))
#                 loss = []

#         best = evaluate(sess, model, "dev", dev_manager, id_to_tag, logger, reset_best_socre=False)
#         if best:
#             save_model(sess, model, FLAGS.ckpt_path, logger)
#         evaluate(sess, model, "test", test_manager, id_to_tag, logger, reset_best_socre=False)

0
1
step:100 loss:23.867287
2


KeyboardInterrupt: 